In [1]:
!pip install bitsandbytes

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline, TorchAoConfig
# import ecco

class GPT2SentimentAnalyzer:
    def __init__(self, model_name="openai-community/gpt2-xl"):
        self.model_name = model_name
        self._load_model_and_tokenizer()

    def _load_model_and_tokenizer(self):
        """Loads the GPT model and tokenizer."""
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype="float16",
            bnb_4bit_use_double_quant=True
        )

        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            quantization_config=quantization_config,
            device_map="auto"
        )
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

    def get_sentiment(self, text):
        """
        Analyzes the sentiment of the given text using the loaded GPT model.
        Args:
            text (str): The meeting transcript to analyze.
        Returns:
            int or None: The sentiment score (1-10) or None if parsing fails.
        """

        system_prompt = (
            """You are an experienced financial analyst; analyze the text between {} carefully
            and assign a sentiment score to it, from 1 representing the most negative to 10
            representing the most positive. Provide only a single number as your answer, please.
            The sentiment is used to predict how the market might react to the earnings calls."""
        )

        # Insert text into {} as per the prompt design
        prompt = f"{system_prompt}\n{{{text}}}"

        # Tokenize and generate
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        outputs = self.model.generate(
            **inputs,
            max_new_tokens=3,
            do_sample=False  # Greedy decoding to avoid randomness
        )

        # Extract and parse the numeric response
        decoded = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        number_candidates = [int(s) for s in decoded.split() if s.isdigit()]
        score = number_candidates[-1] if number_candidates else None

        return score

    # def interpret_with_ecco(self, text):
    #     """
    #     Interprets the prediction using ecco.
    #     Args:
    #         text (str): The meeting transcript to analyze.
    #     Returns:
    #         ???: The output of ecco.
    #     """

    #     lm = ecco.from_pretrained(self.model_name)
    #     output1 = lm.generate(text, generate=1, do_sample=False)
    #     output2 = lm.generate(text, generate=1, do_sample=False, attribution=['ig'])
    #     return output1, output2



if __name__ == "__main__":
    analyzer = GPT2SentimentAnalyzer()

    # Todo: for loop for the whole dataset

    text = "The earnings report shows a strong growth outlook for next year."

    sentiment = analyzer.get_sentiment(text)
    print(f"Sentiment Score: {sentiment}")

    # ecco_output1, ecco_output2 = analyzer.interpret_with_ecco(text)
    # ecco_output1.rankings()
    # ecco_output2.primary_attributions(attr_method='ig', ignore_tokens=[0,1,2,3,4,5,6,43,44])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sentiment Score: 10
